<a href="https://colab.research.google.com/github/kelvinmw1/Kenya_health_facilities_data/blob/main/kenya_health_facility_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#we mount the google drive where the file is stored

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
#import the libraries to be used in the data cleaning

import pandas as pd
import numpy as np

In [51]:
#we now read our file from its location
#check if the file is readable and that all rows and columns are there

df = pd.read_excel("/content/drive/MyDrive/kenya_health_facilities_data_2023.xls")
df.head(5)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,Facility Code,Facility Name,Province,County,District,Division,Type,Owner,Location,Sub Location,...,IPD,OPD,OUTREACH,PMTCT,RAD/XRAY,RHTC/RHDC,TB DIAG,TB LABS,TB TREAT,YOUTH
0,19224,CDF Kiriari Dispensary,Eastern,Embu,Manyatta,Manyatta,Dispensary,Ministry of Health,Ruguru,Ruguru,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19310,St Jude's Huruma Community Health Services,Nairobi,Nairobi,Mathare,Huruma,Medical Clinic,Private Practice - Unspecified,Huruma,Huruma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14180,10 Engineer VCT,Rift Valley,Laikipia,Laikipia East,Central,Dispensary,Armed Forces,Nanyuki,Majengo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17486,12 Engineers,Central,Kiambu,Thika West,NaN,Dispensary,Ministry of Health,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18393,3Kl Maternity & Nursing Home,Rift Valley,Kajiado,Kajiado North,Ongata Rongai,Nursing Home,Private Practice - Clinical Officer,Gataka,Gataka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#check if the data has missing values eg missing county names make the data useless
#determine which missing values might skew your data

df.isnull().sum()


,0
Facility Code,0
Facility Name,0
Province,0
County,0
District,0
Division,267
Type,0
Owner,0
Location,495
Sub Location,892


In [53]:
#check for any duplicates in your data

df.duplicated().sum()

np.int64(0)

In [54]:
#rename your column names so that they are consistent and have no spaces

df.columns = df.columns.str.lower().str.replace(' ', '_')

In [55]:
#replace all the missing data in beds with 0, because some facilities might not have beds and that will be okay for the data unlike missing values

df['beds'] = df['beds'].fillna(0)

In [56]:
#check if all your missing values have been filled

df.isnull().sum()

,0
facility_code,0
facility_name,0
province,0
county,0
district,0
division,267
type,0
owner,0
location,495
sub_location,892


In [57]:
#since we have determined that the hospital ownership data is not standardised because there are 26 different owners, we now standardise it to make the data more accurate to 7 main owners

ownership_map = {

    #Public
    "Ministry of Health": "Public",
    "Local Authority": "Public",
    "Other Public Institution": "Public",
    "Parastatal": "Public",
    "State Coorporation": "Public",
    "Community Development Fund": "Public",
    "Local Authority T Fund": "Public",

    # Private

    "Private Enterprise (Institution)": "Private",
    "Private Practice - Nurse / Midwife": "Private",
    "Private Practice - Clinical Officer": "Private",
    "Private Practice - General Practitioner": "Private",
    "Private Practice - Medical Specialist": "Private",
    "Private Practice - Unspecified": "Private",
    "Company Medical Service": "Private",

    # Faith-Based

    "Kenya Episcopal Conference-Catholic Secretariat": "Faith-Based",
    "Christian Health Association of Kenya": "Faith-Based",
    "Supreme Council for Kenya Muslims": "Faith-Based",
    "Other Faith Based": "Faith-Based",

    # NGO / Humanitarian

    "Non-Governmental Organizations": "NGO",
    "Humanitarian Agencies": "NGO",

    # Community

    "Community": "Community",

    # Military

    "Armed Forces": "Military",

    # Academic

    "Academic (if registered)": "Academic"
}

df['ownership_standardized'] = (
    df['owner']
    .map(ownership_map)
    .fillna("Other")
)


In [58]:
#let us confirm if it works

df['ownership_standardized'].value_counts()

,count
ownership_standardized,
Public,4943
Private,3938
Faith-Based,1098
NGO,356
Community,109
Academic,37
Military,24


In [59]:
#we now standardise the facility type, from 23 different types to 7 distinct facility types for easier data modelling

type_map = {
    # Dispensary
    "Dispensary": "Dispensary",

    # Health Centre
    "Health Centre": "Health Centre",

    # Hospital
    "Other Hospital": "Hospital",
    "Sub-District Hospital": "Hospital",
    "District Hospital": "Hospital",
    "Provincial General Hospital": "Hospital",
    "National Referral Hospital": "Hospital",

    # Clinic
    "Medical Clinic": "Clinic",
    "Medical Centre": "Clinic",
    "Dental Clinic": "Clinic",
    "Eye Clinic": "Clinic",

    # Specialized Facility
    "Nursing Home": "Specialized Facility",
    "VCT Centre (Stand-Alone)": "Specialized Facility",
    "Laboratory (Stand-alone)": "Specialized Facility",
    "Maternity Home": "Specialized Facility",
    "Eye Centre": "Specialized Facility",
    "Radiology Unit": "Specialized Facility",
    "Regional Blood Transfusion Centre": "Specialized Facility",
    "Blood Bank": "Specialized Facility",
    "Funeral Home (Stand-alone)": "Specialized Facility",

    # Administrative / Training
    "District Health Office": "Administrative / Training",
    "Training Institution in Health (Stand-alone)": "Administrative / Training",
    "Rural Health Training Centre": "Administrative / Training",
    "Health Programme": "Administrative / Training",
    "Health Project": "Administrative / Training",

    # Other
    "Not in List": "Other"
}
df['type_standardized'] = (
    df['type']
    .map(type_map)
    .fillna("Other")
)



In [60]:
df['type_standardized'].value_counts()

,count
type_standardized,
Dispensary,4820
Clinic,3429
Health Centre,1126
Hospital,552
Specialized Facility,521
Administrative / Training,38
Other,19


In [61]:


df['ownership_standardized'].value_counts(dropna=False)
df['type_standardized'].value_counts(dropna=False)


,count
type_standardized,
Dispensary,4820
Clinic,3429
Health Centre,1126
Hospital,552
Specialized Facility,521
Administrative / Training,38
Other,19


In [72]:
#our focus is on a county level, so let us see if the county information is correct but first we standardise the naming and check if the names are correct.
#check if there are 47 in total, else there is a problem


df["county"] = df["county"].str.strip().str.title()
df["county"].nunique()



47

In [73]:
#check for spelling mistakes

df["county"].value_counts()

,count
county,
Nairobi,942
Kiambu,506
Meru,461
Nyeri,453
Nakuru,436
Kitui,421
Machakos,374
Mombasa,320
Kajiado,313


In [76]:
#we check to see how facilities are distributed across the counties

facility_by_county = df.groupby(["county","type_standardized"]).size().reset_index(name="facility_count")

facility_by_county

,county,type_standardized,facility_count
0,Baringo,Administrative / Training,1
1,Baringo,Clinic,14
2,Baringo,Dispensary,190
3,Baringo,Health Centre,23
4,Baringo,Hospital,7
...,...,...,...
250,Wajir,Specialized Facility,4
251,West Pokot,Clinic,15
252,West Pokot,Dispensary,104
253,West Pokot,Health Centre,8


In [80]:
#we check to see facility ownership distribution across counties

ownership_by_county = df.groupby(["county","ownership_standardized"]).size().reset_index(name="ownership_counts")

ownership_by_county

,county,ownership_standardized,ownership_counts
0,Baringo,Community,11
1,Baringo,Faith-Based,12
2,Baringo,NGO,2
3,Baringo,Private,18
4,Baringo,Public,194
...,...,...,...
226,West Pokot,Community,2
227,West Pokot,Faith-Based,20
228,West Pokot,NGO,3
229,West Pokot,Private,16


In [83]:
#let us see which counties have no hospitals across counties

hospital_counts = (df[df["type_standardized"] == "hospital"].groupby("county").size().reset_index(name="hospital_count"))

hospital_counts

,county,hospital_count


In [84]:
#let us create extra tables for data visualisation

facility_by_county.to_csv('facility_by_county.csv', index=False)
ownership_by_county.to_csv('ownership_by_county.csv', index=False)
